In [ ]:
# cleaner avoid to restart kernel for each code modification, use it just when alone
from cleaner import modules_cleaner ; modules_cleaner()
# load current project if secondary ipynb runned alone
import metys; metys.Metys.secondary()

In [ ]:
%matplotlib inline

import os
import re
import sys

import cPickle as pickle

from mbt.tools.faults import rates_for_double_truncated_mfd
from openquake.hazardlib.geo.surface import SimpleFaultSurface
from openquake.hazardlib.mfd import EvenlyDiscretizedMFD

from openquake.hazardlib.scalerel.wc1994 import WC1994

In [ ]:
try:
    plot_figures
except: 
    plot_figures = False
    
try:
    print_log
except: 
    print_log = False
    
try:
    len(shallow_fault_source_ids_list)
except:
    raise ValueError('List of source IDs badly defined: ')

In [ ]:
if print_log or plot_figures:
    print '------------------------------------------------------'

# Width of bins used to discretise the MFDs
model_key = metys.g_prj.models.current 
binwidth = float(metys.g_prj.mod[model_key]['mfd_binwidth'])
if print_log:
    print 'MFD bin width    : %.2f' % (binwidth)

# Load sources
model_key = metys.g_prj.models.current 
data_path = os.path.join(metys.g_prj.folder, '%s_fault_sources.pkl' % model_key)

fin = open(data_path,'rb') 
fsources = pickle.load(fin)
fin.close()
    
if len(shallow_fault_source_ids_list) > 1:
    raise ValueError('This notebook can handle just one source')
    
src = fsources[shallow_fault_source_ids_list[0]]

## Parameters

In [ ]:
scalerel_name = metys.g_prj.mod[model_key]['sfs_default_scalerel']
if hasattr(src, 'magnitude_scaling_relationship'):
    scalerel_name = src.magnitude_scaling_relationship

In [ ]:
import importlib
module = importlib.import_module('openquake.hazardlib.scalerel')
my_class = getattr(module, scalerel_name)
scalrel = my_class()

## Computing the mfd

In [ ]:
bin_width = float(metys.g_prj.mod[model_key]['mfd_binwidth'])

# Setting the minimum magnitude 
try:
    m_low = src.min_mag
except:
    m_low = float(metys.g_prj.mod[model_key]['sfs_default_mmin'])

In [ ]:
if src.source_type == 'SimpleFaultSource':
    
    if src.__dict__.has_key('use_slip') and src.use_slip == 1:
        print 'For this source we do not use slip' 
        
    else:       
        
        # Fix upper seismogenic depth
        if not src.__dict__.has_key('upper_seismogenic_depth'):
            upper_seismogenic_depth = float(metys.g_prj.mod[model_key]['sfs_default_upper_seismogenic_depth'])
        else:
            upper_seismogenic_depth = src.upper_seismogenic_depth
            
        # Mesh spacing
        if not src.__dict__.has_key('mesh_spacing'):
            mesh_spacing = float(metys.g_prj.mod[model_key]['sfs_default_mesh_spacing'])
        else:
            mesh_spacing = src.mesh_spacing      
            
        # Fix lower seismogenic depth
        if not src.__dict__.has_key('lower_seismogenic_depth'):
            lower_seismogenic_depth = float(metys.g_prj.mod[model_key]['sfs_default_lower_seismogenic_depth'])
        else:
            lower_seismogenic_depth = src.lower_seismogenic_depth  
            
        # Compute area source
        surf = SimpleFaultSurface.from_fault_data(
                    fault_trace = src.trace, 
                    upper_seismogenic_depth = upper_seismogenic_depth, 
                    lower_seismogenic_depth = lower_seismogenic_depth, 
                    dip = src.dip, 
                    mesh_spacing = mesh_spacing)
        flt_area = surf.get_area()

        # Median magnitude
        if 'mmax' in src.__dict__ and float(getattr(src, 'mmax')) > 0.0:
            median_magnitude = float(getattr(src, 'mmax'))
            print 'Maximum magnitude (from input)               : %.2f' % (median_magnitude)
        else:
            median_magnitude = scalrel.get_median_mag(flt_area, src.rake)
            std = scalrel.get_std_dev_area(median_magnitude, src.rake)
            mmax = median_magnitude + std * 2
            print 'Maximum magnitude (from scaling relationship): %.2f' % (median_magnitude)
         
        print median_magnitude, m_low+bin_width, src.sliprate
        if median_magnitude > m_low+bin_width and src.sliprate > 1e-10:
        
            # Get rates
            if hasattr(src, 'id_intersecting_area'):
                print 'Using parameters of area source ID           : %s' % (src.id_intersecting_area)
                b_gr = model.sources[src.id_intersecting_area].b_gr
            else:
                b_gr = float(metys.g_prj.mod[model_key]['sfs_default_b_gr'])
            
            if 'creeping_coeff' in src.__dict__:
                coeff = 1.-float(getattr(src, 'creeping_coeff'))
                print 'Creeping Coefficient (from input)            : %.2f' % (1.-coeff)
            else:
                coeff = 1.-float(metys.g_prj.mod[model_key]['sfs_default_creeping_coefficient'])
                print 'Creeping Coefficient (default)               : %.2f' % (1.-coeff)
            
            rates = rates_for_double_truncated_mfd(flt_area, 
                                                   src.sliprate*coeff, 
                                                   m_low, 
                                                   median_magnitude, 
                                                   b_gr=b_gr,
                                                   bin_width=bin_width)
            
            if any(rates):
                src.mfd = EvenlyDiscretizedMFD(m_low+bin_width/2, bin_width, rates)
                src.scalerel = scalerel_name
                print 'Computed MFD\n',src.mfd.get_annual_occurrence_rates()
            else:
                print 'MFD not defined: rates not defined'
                src.mfd = None
                src.scalerel = None                
        else:  
            print 'MFD not defined: mag from scaling relationship < mmin + bin_width'
            src.mfd = None
            src.scalerel = None

sources[shallow_fault_source_ids_list[0]] = src 